In [1]:
# Install and import the required packages:
import sys
!conda install --yes --prefix {sys.prefix} biopython #biopython for alignment and revcom
import csv #parses csv
import numpy as np 
from itertools import zip_longest as zip #used to calculate Hamming distance fast
from Bio.Seq import Seq #BioTools sequencing
from Bio import Align #Used for alignment
import pandas as pd #Used for dataframe to CSV

Solving environment: done

# All requested packages already installed.



In [2]:
#Define relevant variables:

#Files:
FileName =  'GUIDEU2OS_FilteredSRv2.txt' # This is where the hits .txt file is given
GuideList = 'Pilot4GUIDEU2OS.csv' # This is where the guides in the form of a .csv are given
OutputFileName = 'Processed-GUIDEU2OS_FilteredSRv2.csv' #This is the name you want for the output file

#Parameters:
NumSamples = 6 #Number of samples
StringSearchLength = 25 #Distance on either side of cut site in target string 
CutDist = 17 #Approximate distance from start of gRNA to cut site
MismatchThreshold = 6 #Maximum number of mismatchs at a real site
MaxPeakThreshold = 1/3 #Minimum Threshold of reads for a multiple peak within a region compared to largest peak

#Delimited Spacer Sequence
DelimiterReadout = True #True for GUIDE-seq-like readout, False for just lowercase
Delimiter = "-" #  Either "‒" or "."

#TTISS-predicted Indel Distribution:
PredictedIndels = False #True if we want to output predicted indels
HistosThreshold = 50 #Minimum number of reads on either side in order to calculate distribution

In [3]:
#Create new numpy array to store values in parsed format

#Parse hits file and store entries in OriginalData Numpy array:
OriginalData = []
with open('GeneralTTISSMatcherHits/'+FileName, 'r') as f:
    reader = csv.reader(f, dialect='excel', delimiter='\t')
    for row in reader:
        OriginalData.append(row)        
OriginalData = np.array(OriginalData)

#First, calculate number of samples and initialize new structure
(rows, columns) = OriginalData.shape 
NumSites = int((rows-1)/(NumSamples+1))
FilteredData = np.zeros((NumSites+1, NumSamples+3),dtype = object)

#Fill out the first row
FilteredData[0,0] = OriginalData[0,0]
FilteredData[0,1] = "Sequence"
for r in range(NumSamples):
        FilteredData[0,r+2] = OriginalData[r+2,0]
FilteredData[0,NumSamples+2]="Summed"

#Build the remaining rows 
for n in range(NumSites):
    FilteredData[n+1,0] = OriginalData[(NumSamples+1)*n+1,0] #Added chromosome position to table
    Sequence = ""
    for n1 in range(60):
        Sequence += OriginalData[(NumSamples+1)*n+1,n1+1]
    FilteredData[n+1,1] = Sequence
    SummedArray = np.zeros((1,120),dtype=int)
    for n2 in range(NumSamples):
        PeakCount = OriginalData[(NumSamples+1)*n+1+n2+1,1:].astype(int)
        SummedArray += PeakCount
        FilteredData[n+1,2+n2]=PeakCount
    FilteredData[n+1,NumSamples+2]=SummedArray

In [4]:
#Create arrays to store guide sequences and match guides to numbers

#Parse guides file and store entries in original guides NP array:
OriginalGuides = []
with open('GeneralTTISSMatcherGuides/'+GuideList, 'r') as f:
    reader = csv.reader(f)
    for row in reader:
        OriginalGuides.append(row)        
OriginalGuides = np.array(OriginalGuides)

#Create list containing all guides and their reverse complements:
Guides =  []
(ro, col) = OriginalGuides.shape
for r in range (ro):
    Guides.append(str(OriginalGuides[r, 1]))
    seq = Seq(OriginalGuides[r, 1])
    Guides.append(str(seq.reverse_complement()))

In [5]:
#Build subroutine to match a given sequences to gRNAs

#Define helper function for Hamming distance:
def hamdist(str1, str2):    
    '''
    Inputs: 
        str1: String of some length
        str2: Another string of the same length as str1
        
    Outputs: 
        Returns Hamming distance, which is the number of differences
        between the two strings
    ''' 
    diffs = 0
    for ch1, ch2 in zip(str1, str2):
        if ch1 != ch2:
            diffs += 1
    return diffs

#Define helper function that takes in the index of a gRNA string from OriginalGuides and outputs gRNA # and sequence
def gRNAfinder(gRNAnumber):
    '''
    Inputs:
        gRNAnumber: index of gRNA sequence from OriginalGuides
        
    Outputs:
        GuideNumber: number of guide from first column of OriginalGuides
        RC: Boolean where True means reverse complemented guide and False means correct strand
        Sequence: normal strand sequence of original gRNA
    '''
    RC = False
    if gRNAnumber%2 == 1: #If matched guide is the RC of a real guide
        GuideNumber = int(OriginalGuides[int((gRNAnumber-1)/2),0])
        Sequence = OriginalGuides[int((gRNAnumber-1)/2),1]
        RC = True
    else:
        GuideNumber = int(OriginalGuides[int(gRNAnumber/2),0])
        Sequence = OriginalGuides[int(gRNAnumber/2), 1]
    '''
    #Test cases:
    print(gRNAfinder(0))
    print(gRNAfinder(1))
    print(gRNAfinder(2))
    print(gRNAfinder(3))
    '''
    
    return [GuideNumber, RC, Sequence]



#Define helper function that takes in a RealString and a gRNA sequence and changes RealString to Delimiter or Lowercase
def StringChanger(RealSpacer, guideString):
    realseq = RealSpacer
    for i in range(20):
        if DelimiterReadout:
            if realseq[i] == guideString[i]:
                newstring = realseq[:i]+Delimiter+realseq[i+1:] 
                realseq = newstring  
        else:
            if realseq[i] != guideString[i]:
                newstring = realseq[:i]+realseq[i].lower()+realseq[i+1:] #slices to make letter lowercase
                realseq = newstring   
                
    '''
    #Test cases:
    DelimiterReadout = False
    print(StringChanger("AAAAAATTTTTTTTTTTTTTTTT", "TAAAAAATTTTTTTTTTTTT"))
    DelimiterReadout = True
    print(StringChanger("AAAAAATTTTTTTTTTTTTTTTT", "TAAAAAATTTTTTTTTTTTT"))
    '''
    
    return realseq



#Define function to match string with gRNA:
def Matcher(TargetString, CutSite):
    '''
    Inputs:
        TargetString: string of length 2*StringSearchLength
        CutSite: Index of predicted cut site, in the middle of target string and a distance CutDist within the gRNA
        Implicitly, the gRNA list Guides and the numbers in OriginalGuides are used as well as MismatchThreshold
    
    Outputs:
        SingleMatch: Boolean with True meaning only a single match under mismatch treshold, False if otherwise
        MM: Mismatch number between hit(s) with original gRNA(s)
        GuideMatch: Number(s) of guide(s) that matched
        CrudeSpacer: string(s) within TargetString that matched under the mismatch threshold or best match
        RealSpacer: string(s) with sequence in the regular direction of the gRNA with MMs either lowercase
                    or matches replaced by the delimiter set in the settings
        CutSiteScore: score(s) for how close match(es) is(are) to predicted cut sites

        
    Possible Errors:
        Match PAM is not in the sequence provided
    '''
    
    #Intialize variables for guide matching
    BestDist = 25
    GuideNumber = 0
    Index = 0 
    CutSiteScore = 100
    
    SingleMatch = True
    MMs = []
    GuideNumberList = []
    CrudeSpacerList = []
    RealSpacerList = []
    CutSiteScoreList = []

    #Iterate over all guides to find best match(es)
    for g in range(len(Guides)):        
        #Iterate over all possible 20 base pair windows
        SpacerBestDist = 25
        SpacerBestMatch = 0
        for m in range(len(TargetString)-20):            
            CurrentString = TargetString[m:m+20] #Defines current test window
            SpacerTestDist = hamdist(CurrentString, Guides[g]) # Calculates Hamdist between test window and current guide
            if SpacerTestDist <= SpacerBestDist:
                SpacerBestDist = SpacerTestDist
                SpacerBestMatch = m
        t=SpacerBestMatch
        TestDist = SpacerBestDist
        #Precomputations for gRNA crude and real spacer
        if TestDist<= MismatchThreshold or TestDist <= BestDist:
            GuideNumber = gRNAfinder(g)[0]
            if gRNAfinder(g)[1] == False: #Not RC
                if TargetString[t:t+23] != "":
                    AlignedString = TargetString[t:t+23]
                    RealString = AlignedString
                else: #covers the corner case where PAM is not in the string
                    AlignedString = TargetString[t:t+20]+" PAM INDEX ERROR "+str(t)
                    RealString = TargetString[t:t+20]
                CutSiteScore = t + CutDist - CutSite  
            else:
                if TargetString[t-3:t+20] != "":
                    AlignedString = TargetString[t-3:t+20]
                    SpacerSeq = Seq(TargetString[t-3:t+20])
                    RealString = str(SpacerSeq.reverse_complement())
                else: #covers the corner case where PAM is not in the string
                    AlignedString = TargetString[t:t+20]+" PAM INDEX ERROR RC "+str(t)
                    SpacerSeq = Seq(TargetString[t:t+20])
                    RealString = str(SpacerSeq.reverse_complement())
                CutSiteScore = t + 20 - CutDist - CutSite  
            RealString = StringChanger(RealString, gRNAfinder(g)[2]) #Convert real string to good format

        '''
        5 cases here: 
            1) TestDist is > MismatchThreshold and > BestDist so no change
            2) TestDist is > MismatchThreshold and < BestDist so BestDist and all variables updated/replaced
            3) TestDist is > MismatchThreshold and = BestDist so BestDist and all variables appended
            4) TestDist is <= MismatchThreshold and BestDist > MismatchThreshold so all variables updated/replaced
            5) TestDist is <= MismatchThreshold and BestDist <= MismatchThreshold so all variables appended
        We are done with Case 1 (no change), and will deal with cases 3 and 5 first
        '''

        #Case 3 and 5
        if (TestDist <= MismatchThreshold and BestDist <= MismatchThreshold) or (TestDist == BestDist):
            SingleMatch = False #Update SingleMatch to reflect multiple matches
            MMs.append(TestDist)
            GuideNumberList.append(GuideNumber)
            CrudeSpacerList.append(AlignedString)
            RealSpacerList.append(RealString)
            CutSiteScoreList.append(CutSiteScore)

        #Case 2 and 4 (BestDist is now implictly > MismatchThreshold if first condition satisfied)
        elif (TestDist <= MismatchThreshold) or (TestDist < BestDist):
            BestDist = TestDist #Update best distance, it is now <= MismatchThreshold
            SingleMatch = True #Update SingleMatch to reflect single match 
            MMs = [TestDist]
            GuideNumberList = [GuideNumber]
            CrudeSpacerList = [AlignedString]
            RealSpacerList = [RealString]
            CutSiteScoreList = [CutSiteScore]      
            
    '''
    #Test Cases
    match1 = Matcher('CTCTTCCCCAGCCCAGCCTCTTCCTGGAGTTCCTTCCTGGGACCTGCAGTGCCAAGTGCT', 30)
    print(match1)
    print(match1[0])
    for i in range(5):
        print(match1[i+1][0])
    print(Matcher('AAATATTCCTTCTGTTCCTTTCTTTCTTTCTTCTCCTGCTGGGACTCCTATTTTATATAT', 30))
    print(Matcher('GAGGCTCAGGCAAGCCGCGCTGAGCCTTTCTTCTCCTTCCCGGACTCCTCACCACCCATC', 30))
    '''
    
    return [SingleMatch, MMs, GuideNumberList, CrudeSpacerList, RealSpacerList, CutSiteScoreList]

In [6]:
#Build subroutine to find (a) peak(s) given aggregate AS and S peak histos
def PeakFinder(peakhistosum):
    '''
    Inputs:
        peakhistosum: aggregated AS and S reads from all tracks
            likely of form FilteredData[i,2+NumSamples][0]
    Outputs:
        FinalIndices: List of predicted cut site positions 
        
    Assumptions: 
        If there are multiple peaks, then other peaks will have convo sum at least MaxPeakThreshold of top peak
        and at least 3 reads. We assume that convolution filter will be enough to pick up peaks
        Things within a 6 bp window of either side of a main peak are also the same cut
    '''
    #First break up peakhistosum into sense and antisense arrays
    peakhistosum = np.array(peakhistosum)
    Shistos = peakhistosum[0:60]
    AShistos = peakhistosum[60:]
    
    #Initialize arrays
    Sindex = []
    ASindex = []
    Indices = []
    FinalIndices = []
    
    #Define filters for convolution and attempted matched filtering
    Sfilter = [10, 50, 100]
    ASfilter = [100, 50, 10]
    
    #Convolve histos with filters
    Sconvo = np.convolve(Shistos, Sfilter, 'valid')
    ASconvo = np.convolve(AShistos, ASfilter, 'valid')
    
    #If there are reads on a given side, find first max peak
    if np.count_nonzero(Sconvo)>0:
        maxSpeak = np.amax(Sconvo)
        maxSindex = np.where(Sconvo == maxSpeak)
        for m in range(len(maxSindex)):
            Sindex.append(maxSindex[m][0]) 
            #Replace 5 on either side of peak with 0
            for r in range(21):
                try:
                    Sconvo[maxSindex[m]-10+r]=0
                except Exception:
                    pass
        #Reset all convolution sums < MaxPeakThreshold of max to 0
        Sconvo[Sconvo<maxSpeak*MaxPeakThreshold]=0
        Sconvo[Sconvo<500]=0
    else:
        Sindex.append("None")
        
    if np.count_nonzero(ASconvo)>0:
        maxASpeak = np.amax(ASconvo)
        maxASindex = np.where(ASconvo == maxASpeak)
        for n in range(len(maxASindex)):
            ASindex.append(maxASindex[n][0]+2) 
            #Replace 5 on either side of peak with 0
            for r in range(21):
                try:
                    ASconvo[maxASindex[n]-10+r]=0
                except Exception:
                    pass
        #Reset all convolution sums < MaxPeakThreshold of max to 0
        ASconvo[ASconvo<maxASpeak*MaxPeakThreshold]=0
        ASconvo[ASconvo<500]=0
    else:
        ASindex.append("None")
    
    #If there are any peaks remaining, store them:
    if np.count_nonzero(Sconvo)>0:
        maxSindex = np.where(Sconvo > 0)
        for m in range(len(maxSindex)):
            if Sconvo[maxSindex[m][0]]!=0:
                Sindex.append(maxSindex[m][0]) 
                #Replace 5 on either side of peak with 0
                for r in range(21):
                    try:
                        Sconvo[maxSindex[m]-10+r]=0
                    except Exception:
                        pass
    if np.count_nonzero(ASconvo)>0:
        maxASindex = np.where(ASconvo > 0)
        for n in range(len(maxASindex)):
            if ASconvo[maxASindex[n][0]]!=0:
                ASindex.append(maxASindex[n][0]+2) 
                #Replace 5 on either side of peak with 0
                for r in range(21):
                    try:
                        ASconvo[maxASindex[n]-10+r]=0
                    except Exception:
                        pass
                
    #We now have two lists of indices, one generated by sense reads and the other by antisense reads. 
    #We now need to merge them. Any indices within 5 of each other are grouped together.
    if Sindex != ["None"] and ASindex !=["None"]:
        Indices = Sindex+ASindex
        Indices = sorted(Indices)
    elif ASindex !=["None"]:
        Indices = sorted(ASindex)
    elif Sindex !=["None"]:
        Indices = sorted(Sindex)
    #Bad coding practice, but we assume that we were not given an array of 0s... so no edge case here
    
    FinalIndices.append(Indices[0])
    lastadded = Indices[0]
    for i in range(len(Indices)-1): 
        if Indices[i+1]-lastadded>10:
            FinalIndices.append(Indices[i+1])
            lastadded = Indices[i+1]
            
    '''
    #Test Cases

    for i in range(50):
        print(np.array(FilteredData[i+1,2+NumSamples][0]))
        print(PeakFinder(FilteredData[i+1,2+NumSamples][0]))
    '''
    return FinalIndices

In [7]:
#We can now go row-by-row (site-by-site) through FilteredData and calculate all we need to, storing into FinalData 

#First, let's calculate the total number of predicted cut sites and their indices
IndiceList = []
PutativeSiteCounter = 0
for a in range(NumSites):
    NewIndices = PeakFinder(FilteredData[a+1,2+NumSamples][0])
    PutativeSiteCounter += len(NewIndices)
    IndiceList.append(NewIndices)

#Intialize and fill in first row of our final structure:
if PredictedIndels:
    pass #TODO LATER
    #Need to integrate from previous code
else:
    FinalData = np.ndarray(shape=(PutativeSiteCounter+1, NumSamples+9), dtype=object)
    FinalData[0,0] = "Genome Position"
    for i in range(NumSamples+2):
        FinalData[0,i+1]=FilteredData[0,i+1]
    FinalData[0, NumSamples+3] = "Single Match?"
    FinalData[0, NumSamples+4] = "MMs"
    FinalData[0, NumSamples+5] = "gRNA"
    FinalData[0, NumSamples+6] = "Crude Spacer"
    FinalData[0, NumSamples+7] = "Real Spacer"
    FinalData[0, NumSamples+8] = "Cut Site Score"

#Now, we iterate through all predicted cut sites and fill in that row:
AnalyzedCutSiteCounter = 0 

for a in range(NumSites): #Iterating through all original cut sites
    for i in range(len(IndiceList[a])): #Iterating through all peaks at each cut site
        FinalData[AnalyzedCutSiteCounter+1,0]=FilteredData[a+1,0] #Fill in position
        FinalData[AnalyzedCutSiteCounter+1,1]=FilteredData[a+1,1] #Fill in local seq
        for j in range(NumSamples):
            FinalData[AnalyzedCutSiteCounter+1,2+j]=sum(FilteredData[a+1,2+j]) #Fill in summed reads per track
        FinalData[AnalyzedCutSiteCounter+1,2+NumSamples]=sum(FilteredData[a+1,2+NumSamples][0]) #Fill in summed
        #Try to pull out local seq for matching
        CurrentIndex = IndiceList[a][i]
        if CurrentIndex <=StringSearchLength:
            LocalSpacerSeq = FilteredData[a+1,1][0:CurrentIndex+StringSearchLength+1]
            MatchedOutput = Matcher(LocalSpacerSeq, CurrentIndex)
        elif CurrentIndex >=60-StringSearchLength:
            LocalSpacerSeq = FilteredData[a+1,1][CurrentIndex-StringSearchLength:]
            MatchedOutput = Matcher(LocalSpacerSeq, StringSearchLength+1)
        else: 
            LocalSpacerSeq =  FilteredData[a+1,1][IndiceList[a][i]-StringSearchLength:IndiceList[a][i]+StringSearchLength+1]
            MatchedOutput = Matcher(LocalSpacerSeq, StringSearchLength+1)
        FinalData[AnalyzedCutSiteCounter+1, NumSamples+3+0]=MatchedOutput[0]
        if MatchedOutput[0] == True:
            if MatchedOutput[1] != []:
                for k in range(5):
                    FinalData[AnalyzedCutSiteCounter+1, NumSamples+4+k] = MatchedOutput[k+1][0]
            else:
                for k in range(5):
                    FinalData[AnalyzedCutSiteCounter+1, NumSamples+4+k] = "ERROR" 
        else:
            for l in range(5):
                FinalData[AnalyzedCutSiteCounter+1, NumSamples+4+l] = MatchedOutput[l+1]
        AnalyzedCutSiteCounter += 1
        if AnalyzedCutSiteCounter % 50 == 0:
            print("Analyzed "+str(AnalyzedCutSiteCounter)+" out of " + str(PutativeSiteCounter)+" putative cut sites")
    

Analyzed 50 out of 3196 putative cut sites
Analyzed 100 out of 3196 putative cut sites
Analyzed 150 out of 3196 putative cut sites
Analyzed 200 out of 3196 putative cut sites
Analyzed 250 out of 3196 putative cut sites
Analyzed 300 out of 3196 putative cut sites
Analyzed 350 out of 3196 putative cut sites
Analyzed 400 out of 3196 putative cut sites
Analyzed 450 out of 3196 putative cut sites
Analyzed 500 out of 3196 putative cut sites
Analyzed 550 out of 3196 putative cut sites
Analyzed 600 out of 3196 putative cut sites
Analyzed 650 out of 3196 putative cut sites
Analyzed 700 out of 3196 putative cut sites
Analyzed 750 out of 3196 putative cut sites
Analyzed 800 out of 3196 putative cut sites
Analyzed 850 out of 3196 putative cut sites
Analyzed 900 out of 3196 putative cut sites
Analyzed 950 out of 3196 putative cut sites
Analyzed 1000 out of 3196 putative cut sites
Analyzed 1050 out of 3196 putative cut sites
Analyzed 1100 out of 3196 putative cut sites
Analyzed 1150 out of 3196 puta

In [8]:
#Writes final matrix to csv file of choice:
df = pd.DataFrame(FinalData)
df.to_csv('GeneralTTISSMatcherProcessed/'+OutputFileName, index = False, header = False)